In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

from tensorflow.keras import Sequential,layers,losses,utils,Input
from tensorflow.keras.layers import Dense,LSTM,Dropout,concatenate,Flatten, Conv1D, MaxPooling1D,Activation,RepeatVector,TimeDistributed
import tensorflow as tf
# from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional,GRU
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager


from sklearn.decomposition import PCA

In [2]:
data=pd.read_csv('changping.csv')

In [ ]:
data

In [ ]:
data.columns

In [5]:
col=[ 'year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2',
       'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN',  'WSPM','wd']
datacol=data[col]

In [ ]:
datacol.isnull().sum()

In [7]:
# 直接删除缺失值
datashan=datacol.dropna()

In [ ]:
datashan.isnull().sum()

In [ ]:
datashan.shape

In [ ]:
datashan.describe()

In [ ]:
datashan.max()

In [ ]:
# 拼接时间为一列，绘制一年时间降水分布图
# datag['时间(年月日时)']=datag['年(年)'].map(str)+''+datag['月(月)'].map(str)+''+datag['日(日)'].map(str)+''+datag['时(时)'].map(str)
datashan['datatime']=datashan['year'].map(str)+'-'+datashan['month'].map(str)+'-'+datashan['day'].map(str)+'-'+datashan['hour'].map(str)

In [ ]:
datashan.head(8)

In [ ]:
# datag=data.copy()
datashan['datatime']=pd.to_datetime(datashan['datatime'],format='%Y-%m-%d-%H',errors='coerce')
datashan.index=datashan['datatime']

In [ ]:
datashan.isnull().sum()

In [16]:
# # 重采样成日为单位的数据
# data_mean = datashan.resample('D').mean()
# print(data_mean.tail(10))
# data_mean.plot()
# plt.show()

In [17]:
# data_mean.shape

In [18]:
# data_meanshan=data_mean.dropna()

In [16]:
chongcol=[ 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
datag=datashan[chongcol]

In [17]:
datal=datashan[chongcol]

In [ ]:
datag

In [ ]:
# 归一化
scaler = MinMaxScaler(feature_range=(0, 1))  # minmaxscaler 方法，用来做归一化
sel_col = datag.columns
print(sel_col)
for col in sel_col:
    datag[col] = scaler.fit_transform(datag[col].values.reshape(-1, 1))  # reshape成为一列
print("Load dataset LEN: ", datag.shape[0])
print(sel_col)


In [ ]:
datag

In [ ]:
datag.corr()

In [ ]:
X = []
Y = []
columns_name = list(datag.columns)
y_index = columns_name.index('PM2.5')
tmp = np.array(datag, dtype=np.float32)
print("The shape of tmp is:")
print(tmp.shape)
pca = PCA(n_components=0.97)# 保证降维后的数据保持90%的信息
tmp2 = pca.fit_transform(tmp[:,1:])
print("The shape of tmp2 is:")
print(tmp2.shape)  # PCA降维y_index要改，或者先提取出来
print("input dim is:")
print(tmp2.shape[1])

In [26]:
seqX=120
seqY=1
day_delay=1
gap=1
for i in range(tmp2.shape[0] - (seqX+seqY+day_delay)*gap):
        # seqX, seqX+day_delay, seqX+day_delay+seqY
        # format: i+x*gap
        X.append(tmp2[i:(i + seqX*gap):gap, :])  # tmp2是PCA降维后的特征，如果是tmp那就是原始特征
        Y.append(tmp[(i+(seqX+day_delay)*gap): (i+(seqX+day_delay+seqY)*gap): gap, y_index])
X = np.array(X)
Y = np.array(Y)

In [ ]:
X.shape

In [ ]:
X

In [ ]:
Y.shape

In [ ]:
Y

In [29]:
X = X.astype(np.float32)
Y = Y.astype(np.float32)
p = np.random.permutation(range(len(X)))
X,Y = X[p],Y[p]

In [ ]:
testx.shape

In [30]:
trainx, trainy = X[:int(0.8 * 32652)], Y[:int(0.8 * 32652)]  

In [508]:
testx, testy = X[int(0.8 * 32652):], Y[int(0.8 * 32652):] 

In [ ]:
# 将训练集存起来
nptrainx=np.reshape(trainx,(-1,7))
nptrainx

In [ ]:
type(nptrainx)

In [90]:
pdtrainx1=np.reshape(nptrainx,(-1,120,7))

In [ ]:
pdtrainx1

In [95]:
pdtrainx=pd.DataFrame(nptrainx,columns=['t1','t2','t3','t4','t5','t6','t7'])

In [ ]:
pdtrainx

In [ ]:
pdtrainx.shape

In [99]:
pdtrainx.to_csv('trainx1.csv')

In [107]:
pdtrainy=pd.DataFrame(trainy,columns=['y1'])
pdtrainy.to_csv('trainy.csv')

In [ ]:
pdtrainy.shape

In [82]:
# 将测试集存起来
nptestx=np.reshape(testx,(-1,7))

In [100]:
pdtestx=pd.DataFrame(nptestx,columns=['t1','t2','t3','t4','t5','t6','t7'])
pdtestx.to_csv('testx.csv')

In [ ]:
pdtestx.shape

In [106]:
pdtesty=pd.DataFrame(testy,columns=['y1'])
pdtesty.to_csv('testy.csv')

In [ ]:
trainx.shape

In [ ]:
testx.shape

In [ ]:
testy.shape

# 读取保存的数据

In [22]:

# 导入训练集
trainx1=pd.read_csv('trainx1.csv',usecols=['t1','t2','t3','t4','t5','t6','t7'])
trainx2=np.array(trainx1)
trainx=np.reshape(trainx2,(-1,120,7))
# print(trainx.shape)
trainy1=pd.read_csv('trainy.csv',usecols=['y1'])
trainy=np.array(trainy1)
# print(trainy.shape)
#导入训练集
testx1=pd.read_csv('testx.csv',usecols=['t1','t2','t3','t4','t5','t6','t7'])
testx2=np.array(testx1)
testx=np.reshape(testx2,(-1,120,7))
# print(testx.shape)
testy1=pd.read_csv('testy.csv',usecols=['y1'])
testy=np.array(testy1)
# print(testy.shape)

# 以下是验证模型是怎末做的 与整体预测无关

In [ ]:
testx2.shape

In [ ]:
testx2[:100]

In [ ]:
testx

In [ ]:
testx

In [ ]:
testx.shape

In [43]:
import keras

In [50]:
score_first_part = Dense(7, use_bias=False, name='attention_score_vec')(testx)

In [ ]:
score_first_part.shape

In [ ]:
score_first_part[0,0,:]

In [45]:
ht = keras.layers.Lambda(lambda x: x[:, -1, :], output_shape=(7,), name='last_hidden_state')(testx)

In [ ]:
ht[0,:]

In [ ]:
a=np.array([ 0.24792711, -0.40586165,  0.21901774, -0.16826439,  0.00261129,0.1956167 , -0.13794912])
b=np.array([ 0.3186469 , -0.27929923,  0.05506572, -0.16007142,  0.02088785,0.11133514, -0.07434156])
a*b

In [ ]:
7.90012050e-02+1.13356846e-01+1.20603695e-02+2.69343198e-02+5.45442338e-05+2.17790127e-02+1.02553528e-02

In [ ]:
ht.shape

In [55]:
score = keras.layers.Dot(axes=[1, 2], name='attention_score')([ht, score_first_part])

In [ ]:
score

In [57]:
attention_weights = Activation('softmax', name='attention_weight')(score)

In [ ]:
attention_weights

In [63]:
context_vector = keras.layers.Dot(axes=[1, 1], name='context_vector')([testx, attention_weights])

In [ ]:
context_vector

In [101]:
a=np.array([[[1,2],[2,2],[1,2]],[[1,2],[1,2],[1,2]],[[1,2],[3,2],[1,2]]])

In [ ]:
a.shape

In [ ]:
a

In [107]:
b=np.array([[1,2,3],[1,1,2],[2,2,3]])

In [ ]:
b.shape

In [ ]:
keras.layers.Dot(axes=[1,1])([a,b])

In [66]:
pre_activation = keras.layers.Concatenate(name='attention_output')([context_vector, ht])

In [ ]:
pre_activation

In [68]:
attention_vector = Dense(128, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)

In [ ]:
attention_vector

构造训练作用的批数据

In [30]:
# # # 构造批数据
def create_batch_dataset(x,y,train=True,buffer_size=1000,batch_size=64):#buffer_size=1000表示可以打乱窗口里面的数据
    batch_data=tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y)))#数据封装，tensor类型
    if train:
        return batch_data.cache().shuffle(buffer_size).batch(batch_size)
    else:
        return batch_data.batch(batch_size)

In [31]:
train_data_single=create_batch_dataset(trainx, trainy,train=True)
val_data_single=create_batch_dataset(testx,testy,train=False)

In [ ]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,input_shape=trainx.shape[-2:],return_sequences=True))
single_step_model.add(Dropout(0.2))
single_step_model.add(tf.keras.layers.LSTM(64,input_shape=trainx.shape[-2:],return_sequences=True))
single_step_model.add(Dropout(0.2))
single_step_model.add(tf.keras.layers.LSTM(32,input_shape=trainx.shape[-2:],return_sequences=True))
single_step_model.add(Dropout(0.1))
single_step_model.add(tf.keras.layers.LSTM(64,input_shape=trainx.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))
single_step_model.add(tf.keras.layers.Activation('linear'))
single_step_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mae')

single_step_history = single_step_model.fit(train_data_single, epochs=500,
#                                             steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            )#validation_steps=50

In [ ]:
# 预测
single_pres=single_step_model.predict(testx,verbose=1)

In [ ]:
single_pres.shape

In [ ]:
single_pres.min()

In [ ]:
testy.shape

In [ ]:
lstmy_modify1 = single_pres*(single_pres>=0)
lstmr1=r2_score(testy,lstmy_modify1)
lstmr1

In [ ]:
# 计算rmse
np.sqrt(mean_squared_error(labels1,preds1))

测试2

In [ ]:
single_step_model1 = tf.keras.models.Sequential()
single_step_model1.add(tf.keras.layers.Bidirectional(LSTM(32,input_shape=trainx.shape[-2:],return_sequences=True)))
single_step_model1.add(Dropout(0.2))
single_step_model1.add(tf.keras.layers.Bidirectional(LSTM(64,input_shape=trainx.shape[-2:],return_sequences=True)))
single_step_model1.add(Dropout(0.2))
single_step_model1.add(tf.keras.layers.Bidirectional(LSTM(32,input_shape=trainx.shape[-2:],return_sequences=True)))
single_step_model1.add(Dropout(0.1))
single_step_model1.add(tf.keras.layers.Bidirectional(LSTM(64,input_shape=trainx.shape[-2:])))
single_step_model1.add(tf.keras.layers.Dense(1))
single_step_model1.add(tf.keras.layers.Activation('linear'))
single_step_model1.compile(optimizer=tf.keras.optimizers.Adam(), loss='mae')

single_step_history1 = single_step_model1.fit(train_data_single, epochs=500,
#                                             steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            )#validation_steps=50

保存LSTM模型以及history

In [ ]:
single_step_model.save('saveLSTM_model')

In [471]:
pd.DataFrame(single_step_history.history).to_csv('saveLSTM_history.csv', index=False)

绘制LSTM损失图与拟合图

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(single_step_history.history['val_loss'],label=['lstm_loss_value'])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend()
plt.legend(loc='best')
plt.savefig("LSTM损失曲线.jpg")
plt.show()

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)
preds1 = np.array(lstmy_modify1)  # 转换为numpy形式
labels1 = np.array(testy)
# print(preds)
# print(labels.shape)
# print(preds.shape)
preds1 = preds1.reshape((-1,1))
future_len1 = preds1.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(future_len1):
    labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(future_len):
    preds1[:, k] = preds1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
# LSTM拟合图
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('LSTM模型',fontsize=16)
ax.plot(labels1[0:200, 0], "b", label="real", alpha=0.8,ms=10)
ax.plot(preds1[0:200, 0], "r", label="pred", alpha=0.8,ms=10)
plt.xlabel('time',fontsize=16)
plt.ylabel('PM2.5',fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=16)
plt.savefig("LSTM拟合图.jpg")

保存LSTM模型的预测数据

In [ ]:
preds1

In [383]:
lstm_data=pd.DataFrame(preds1,columns=['lstm_pred'])

In [ ]:
lstm_data

In [385]:
lstm_data.to_csv('lstmdata.csv')

读取lstm数据

In [23]:
# 读取histort
lstmdata=pd.read_csv('lstmdata.csv')

In [ ]:
lstmdata

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)
# preds1 = np.array(lstmy_modify1)  # 转换为numpy形式
labels1 = np.array(testy)
# print(preds)
# print(labels.shape)
# print(preds.shape)
# preds1 = preds1.reshape((-1,1))
future_len1 = labels1.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(future_len1):
    labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
# for k in range(future_len):
#     preds1[:, k] = preds1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('LSTM model',fontsize=20)
ax.plot(labels1[0:200, 0], "k:", label="real", alpha=0.8,ms=10)
ax.plot(lstmdata['lstm_pred'][0:200], "k", label="pred", alpha=0.8,ms=10)
plt.xlabel('time/h',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
# plt.xlim(0,None)
# plt.ylim(0,None)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("坐标bLSTM拟合图.jpg")

In [26]:
lstm_history=pd.read_csv('saveLSTM_history.csv')

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(lstm_history['val_loss'],label=['LSTM_loss_value'],color="k")
plt.xlabel('number',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.ylabel('LSTM_loss_value',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
# plt.rcParams.update({'font.size': 14})
# plt.ylim(0,None)
# plt.xlim(0,None)
plt.legend(fontsize=20)
# plt.legend(loc='best')
plt.savefig("坐标lstm损失图.jpg")
plt.show()

读取基于注意力机制的数据

In [35]:
from  keras.models import load_model

In [ ]:
# 读取模型
newbalstm_model = load_model('savezhongattbils_model')

In [ ]:
newbalstm_pred=newbalstm_model.predict(testx,verbose=1)

In [ ]:
r2_score(testy,newbalstm_pred)

In [99]:
# 读取histort
attbils_history=pd.read_csv('savezhongabilstm_history.csv')

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(attbils_history['val_loss'],label=['Bi-LSTM-Attention_loss_value'],color="k")
plt.xlabel('number',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.ylabel('Bi-LSTM-Attention_loss_value',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
# plt.rcParams.update({'font.size': 14})
# plt.ylim(0,None)
# plt.xlim(0,None)
plt.legend(fontsize=20)
# plt.legend(loc='best')
plt.savefig("坐标k94attbilstm损失图.jpg")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(attbils_history['val_loss'],label=['bi-lstm-att_loss_value'],color="k")
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend()
plt.legend(loc='best')
plt.savefig("k94attbilstm损失图.jpg")
plt.show()

In [31]:
# 读取
newbalstm_pred=pd.read_csv('savezhongablstm.csv')

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)
abilstmpreds = np.array(newbalstm_pred)  # 转换为numpy形式
labels1 = np.array(testy)
# print(preds)
# print(labels.shape)
# print(preds.shape)
abilstmpreds = abilstmpreds.reshape((-1,1))
future_len1 = abilstmpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(future_len1):
    labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(future_len1):
    abilstmpreds[:, k] = abilstmpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(labels1,abilstmpreds))

In [ ]:
# LSTM拟合图
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=16)
ax.plot(labels1[0:200, 0], "b", label="real", alpha=0.8,ms=10)
ax.plot(abilstmpreds[0:200, 0], "r", label="pred", alpha=0.8,ms=10)
plt.xlabel('time',fontsize=16)
plt.ylabel('PM2.5',fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=16)
plt.savefig("94attbilstm拟合图.jpg")

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontproperties=font_manager.FontProperties(fname='C:\\Users\\caoyu\\AppData\\Local\\Microsoft\\Windows\\Fonts\\FZSYJW.TTF'),fontsize=20)
ax.plot(labels1[0:200, 0], "k:", label="real", alpha=0.8,ms=10)
ax.plot(abilstmpreds[0:200], "k", label="pred", alpha=0.8,ms=10)
plt.xlabel('time/h',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
# plt.xlim(0,None)
# plt.ylim(0,None)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("坐标bLSTM-attention拟合图.jpg")

In [ ]:
# 黑色图
# LSTM拟合图
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=16)
ax.plot(labels1[0:200, 0], "k:", label="real", alpha=0.8,ms=10)
ax.plot(abilstmpreds[0:200, 0], "k", label="pred", alpha=0.8,ms=10)
plt.xlabel('time',fontsize=16)
plt.ylabel('PM2.5',fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=16)
plt.savefig("b94attbilstm拟合图.jpg")

读取GRU数据

In [86]:
gru_data=pd.read_csv('gru_pred.csv')

In [ ]:
gru_data_modify=gru_data['gru_pred'].values.reshape(-1,1)
gru_data_modify

In [ ]:
# GRU拟合图
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('GRU模型',fontsize=16)
ax.plot(labels1[0:200, 0], "b", label="real", alpha=0.8,ms=10)
ax.plot(gru_data_modify[0:200, 0], "r", label="pred", alpha=0.8,ms=10)
plt.xlabel('time',fontsize=16)
plt.ylabel('PM2.5',fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=16)
plt.savefig("GRU拟合图.jpg")

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('GRU模型',fontproperties=font_manager.FontProperties(fname='C:\\Users\\caoyu\\AppData\\Local\\Microsoft\\Windows\\Fonts\\FZSYJW.TTF'),fontsize=20)
ax.plot(labels1[0:200, 0], "k:", label="real", alpha=0.8,ms=10)
ax.plot(gru_data_modify[0:200, 0], "k", label="pred", alpha=0.8,ms=10)
plt.xlabel('time/h',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
# plt.xlim(0,None)
# plt.ylim(0,None)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("坐标GRU拟合图.jpg")

In [ ]:
# 黑色GRU拟合图 
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('GRU模型',fontsize=16)
ax.plot(labels1[0:200, 0], "k:", label="real", alpha=0.8,ms=10)
ax.plot(gru_data_modify[0:200, 0], "k", label="pred", alpha=0.8,ms=10)
plt.xlabel('time',fontsize=16)
plt.ylabel('PM2.5',fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=16)
plt.savefig("bGRU拟合图.jpg")

读取双向lstm数据

In [113]:
# 读取bilstm数据
bilstmmodel=keras.models.load_model('saveBiLSTM_model')

In [ ]:
bilstmpre=bilstmmodel.predict(testx,verbose=1)

In [ ]:
bilstmy_modify1 = bilstmpre*(bilstmpre>=0)
bilstmr=r2_score(testy,bilstmy_modify1)
bilstmr

In [49]:
# bilstmdata=pd.read_csv('bilstm_model.csv')

In [30]:
# bilstm_data_modify=bilstmdata['bilstm_pred'].values.reshape(-1,1)
# bilstm_data_modify

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)
bilstmpreds = np.array(bilstmy_modify1)  # 转换为numpy形式
labels1 = np.array(testy)
# print(preds)
# print(labels.shape)
# print(preds.shape)
bilstmpreds = bilstmpreds.reshape((-1,1))
future_len1 = bilstmpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(future_len1):
    labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(future_len1):
    bilstmpreds[:, k] = bilstmpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(labels1,bilstmpreds))

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM模型',fontproperties=font_manager.FontProperties(fname='C:\\Users\\caoyu\\AppData\\Local\\Microsoft\\Windows\\Fonts\\FZSYJW.TTF'),fontsize=20)
ax.plot(labels1[0:200, 0], "k:", label="real", alpha=0.8,ms=10)
ax.plot(bilstmpreds[0:200, 0], "k", label="pred", alpha=0.8,ms=10)
plt.xlabel('time/h',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
# plt.xlim(0,None)
# plt.ylim(0,None)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("坐标bi-lstm拟合图.jpg")

In [ ]:
# bilstm拟合图
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM模型',fontsize=16)
ax.plot(labels1[0:200, 0], "k:", label="real", alpha=0.8,ms=10)
ax.plot(bilstmpreds[0:200, 0], "k", label="pred", alpha=0.8,ms=10)
plt.xlabel('time',fontsize=16)
plt.ylabel('PM2.5',fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=16)
plt.savefig("biLSTM拟合图.jpg")

In [110]:
bilstm_data=pd.read_csv('bilstm_history.csv')

In [ ]:
bilstm_data

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(bilstm_data['val_loss'],label=['Bi-LSTM_loss_value'],color="k")
plt.xlabel('number',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.ylabel('Bi-LSTM_loss_value',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
# plt.rcParams.update({'font.size': 14})
# plt.ylim(0,None)
# plt.xlim(0,None)
plt.legend(fontsize=20)
# plt.legend(loc='best')
plt.savefig("坐标BI-LSTM损失图.jpg")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(bilstm_data['val_loss'],label=['bi-lstm-loss_value'],color="k")
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend()
plt.legend(loc='best')
plt.savefig("kbilstm损失图.jpg")
plt.show()

In [35]:
# # 黑色bi-lstm
# # bilstm拟合图
# plt.figure(figsize=(10, 6))
# ax = plt.subplot(111)
# plt.rcParams['font.sans-serif']=['SimHei']
# plt.rcParams['axes.unicode_minus']=False
# plt.grid(color="k", linestyle=":")
# ax.set_title('Bi-LSTM模型模型',fontsize=16)
# ax.plot(labels1[0:200, 0], "k:", label="real", alpha=0.8,ms=10)
# ax.plot(bilstm_data_modify[0:200, 0], "k", label="pred", alpha=0.8,ms=10)
# plt.xlabel('time',fontsize=16)
# plt.ylabel('PM2.5',fontsize=16)
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.rcParams.update({'font.size': 14})
# plt.legend(fontsize=16)
# plt.savefig("bbiLSTM拟合图.jpg")

In [40]:
# # 双向LSTM拟合图
# plt.figure(figsize=(12, 6))
# ax = plt.subplot(111)
# plt.grid(color="k", linestyle=":")
# ax.set_title('bi-lstm-att-pm2.5',fontsize=16)
# ax.plot(labels1[0:200, 0], "b", label="real", alpha=0.8,ms=10)
# ax.plot(bilstm_data_modify[0:200, 0], "r", label="pred", alpha=0.8,ms=10)
# plt.xlabel('time',fontsize=16)
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.rcParams.update({'font.size': 14})
# plt.legend()
# plt.savefig("biLSTM-attj拟合图.jpg")

# 总图

In [ ]:
lstmdata['lstm_pred']

In [ ]:
# 总图
plt.figure(figsize=(10, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
# plt.grid(color="k", linestyle=":")
ax.set_title('四种模型对比',fontproperties=font_manager.FontProperties(fname='C:\\Users\\caoyu\\AppData\\Local\\Microsoft\\Windows\\Fonts\\FZSYJW.TTF'),fontsize=25)
ax.plot(labels1[0:100, 0], "ko:", label="real", alpha=0.8,ms=5)
ax.plot(lstmdata['lstm_pred'][0:100], "b", label="lstm-pred", alpha=0.6,ms=15)
ax.plot(bilstmpreds[0:100, 0], "g", label="bi-lstm-pred", alpha=0.6,ms=16)
ax.plot(gru_data_modify[0:100, 0], "r", label="gru-pred", alpha=0.6,ms=17)
ax.plot(abilstmpreds[0:100, 0], "",  label="bi-lstm-attention-pred", alpha=0.6,ms=17)

plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("坐标总体拟合图.jpg")

GRU

In [ ]:
gru_model = tf.keras.models.Sequential()
gru_model.add(tf.keras.layers.GRU(32,input_shape=trainx.shape[-2:],return_sequences=True))
gru_model.add(Dropout(0.2))
gru_model.add(tf.keras.layers.GRU(64,input_shape=trainx.shape[-2:],return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(tf.keras.layers.GRU(32,input_shape=trainx.shape[-2:],return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(tf.keras.layers.GRU(64,input_shape=trainx.shape[-2:]))
gru_model.add(tf.keras.layers.Dense(1))
gru_model.add(tf.keras.layers.Activation('linear'))
gru_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mae')

gru_history = gru_model.fit(train_data_single, epochs=600,
#                                             steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            )#validation_steps=50

In [ ]:
# 预测
gru_pres=gru_model.predict(testx,verbose=1)

In [ ]:
gruy_modify= gru_pres*(gru_pres>=0)
grur=r2_score(testy,gruy_modify)
grur

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)
grupreds = np.array(gruy_modify)  # 转换为numpy形式
grulabels = np.array(testy)
# print(preds)
# print(labels.shape)
# print(preds.shape)
grupreds = grupreds.reshape((-1,1))
future_len = grupreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(future_len):
    grulabels[:, k] = grulabels[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(future_len):
    grupreds[:, k] = grupreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化


In [333]:
# 将预测值存起来用来画图
grupreds_pd=pd.DataFrame(grupreds,columns=['gru_pred'])

In [ ]:
grupreds_pd

In [335]:
grupreds_pd.to_csv('gru_pred.csv')

In [ ]:
# 计算rmse
np.sqrt(mean_squared_error(grulabels,grupreds))

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(gru_history.history['val_loss'],label=['gru_loss_value'])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend()
plt.legend(loc='best')
plt.savefig("GRU损失曲线.jpg")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
ax = plt.subplot(111)
plt.grid(color="k", linestyle=":")
ax.set_title('gru-pm2.5',fontsize=16)
ax.plot(grulabels[0:200, 0], "b", label="real", alpha=0.8,ms=10)
ax.plot(grupreds[0:200, 0], "r", label="pred", alpha=0.8,ms=10)
plt.xlabel('time',fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend()
plt.savefig("GRU拟合图.jpg")

保存GRU模型以及history

In [ ]:
gru_model.save('saveGRU_model')

In [477]:
pd.DataFrame(gru_history.history).to_csv('saveGRU_history.csv',index=False)

In [84]:
gru_history=pd.read_csv('saveGRU_history.csv')

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(gru_history['val_loss'],label=['GRU_loss_value'],color="k")
plt.xlabel('number',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.ylabel('GRU_loss_value',fontproperties=font_manager.FontProperties(fname='C:\\Windows\\Fonts\\times.ttf'),fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
# plt.rcParams.update({'font.size': 14})
# plt.ylim(0,None)
# plt.xlim(0,None)
plt.legend(fontsize=20)
# plt.legend(loc='best')
plt.savefig("坐标GRU损失图.jpg")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(gru_history['val_loss'],label=['gru_loss_value'],color="k")
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams.update({'font.size': 14})
plt.legend()
plt.legend(loc='best')
plt.savefig("kgru损失图.jpg")
plt.show()

# 以下不用看

加入注意力机制

In [111]:
from keras.layers import *
from keras.models import *

In [ ]:
# #建立模型
# inputs=Input(shape=(120,7))
# lstm_inputs=Permute([2,1])(inputs)
# lstm=LSTM(32,activation='relu', return_sequences=True)(lstm_inputs)
# lstm=Permute([2,1])(lstm)#置换维度
# lstm=Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(lstm)
# attention=Dense(7, activation='sigmoid', name='attention_vec')(lstm)#求解Attention权重
# attention=Activation('linear',name='attention_weight')(attention)
# model=Multiply()([lstm, attention])#attention与LSTM对应数值相乘
# outputs = Dense(1, activation='relu')(model)
# model = Model(inputs=inputs, outputs=outputs)
# model.compile(loss='mse',optimizer='adam',metrics=['mae'])
# model.summary()#展示模型结构

In [140]:
import keras

In [164]:
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, lstm_units)

    # (batch_size, time_steps, lstm_units) -> (batch_size, lstm_units, time_steps)
    a = Permute((2, 1))(inputs)

    # 对最后一维进行全连接
    # (batch_size, lstm_units, time_steps) -> (batch_size, lstm_units, time_steps)
    a = Dense(120, activation='softmax')(a)

    # (batch_size, lstm_units, time_steps) -> (batch_size, time_steps, lstm_units)
    a_probs = Permute((2, 1), name='attention_vec')(a)

    # 相乘
    # 相当于获得每一个step中，每个维度在所有step中的权重
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul


In [187]:
#-------------------------------------------#
#  建立注意力模型
#-------------------------------------------#
def get_attention_model():
    inputs=Input(shape=(120,7))
    lstm_units = 32
    # (batch_size, time_steps, INPUT_DIM) -> (batch_size, input_dim, lstm_units)
    lstm_out = LSTM(lstm_units, return_sequences=True)(inputs)
    attention_mul = attention_3d_block(lstm_out)
    # (batch_size, input_dim, lstm_units) -> (batch_size, input_dim*lstm_units)
    attention_mul = Flatten()(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
#     model = Model(inputs=inputs1, outputs=output)
    model = Model(inputs=[inputs], outputs=output)
    return model

In [188]:
att_model = get_attention_model()

In [189]:
att_model.compile(optimizer='adam', loss='mae', metrics=['mse'])

In [ ]:
att_model_history = att_model.fit(train_data_single, epochs=800,
#                                             steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            )#validation_steps=50

In [ ]:
# 预测
att_lstm_pre=att_model.predict(testx,verbose=1)

In [ ]:
att_lstm_pre.shape

In [ ]:
attlstmy_modify = att_lstm_pre*(att_lstm_pre>=0)
attlstmr=r2_score(testy,attlstmy_modify)
attlstmr

In [ ]:
plt.figure(figsize=(16, 8))
ax = plt.subplot(111)
ax.set_title('pm2.5')
ax.plot(testy[0:300, 0], "b", label="real", alpha=0.6)
ax.plot(attlstmy_modify[0:300, 0], "r", label="pred", alpha=0.6)
plt.xlabel('time')
plt.legend()

In [ ]:
att_model_history2 = att_model.fit(train_data_single, epochs=300,
#                                             steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            )#validation_steps=50

In [ ]:
# 预测
att_lstm_pre2=att_model.predict(testx,verbose=1)

In [ ]:
attlstmy_modify2 = att_lstm_pre2*(att_lstm_pre2>=0)
attlstmr2=r2_score(testy,attlstmy_modify2)
attlstmr2

In [ ]:
plt.figure(figsize=(16, 8))
ax = plt.subplot(111)
ax.set_title('pm2.5')
ax.plot(testy[0:300, 0], "b", label="real", alpha=0.6)
ax.plot(attlstmy_modify2[0:300, 0], "r", label="pred", alpha=0.6)
plt.xlabel('time')
plt.legend()

In [202]:
def get_attention_model2():
    inputs=Input(shape=(120,7))
    lstm_units = 64
    # (batch_size, time_steps, INPUT_DIM) -> (batch_size, input_dim, lstm_units)
    lstm_out = LSTM(lstm_units, return_sequences=True)(inputs)
    attention_mul = attention_3d_block(lstm_out)
    # (batch_size, input_dim, lstm_units) -> (batch_size, input_dim*lstm_units)
    attention_mul = Flatten()(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
#     model = Model(inputs=inputs1, outputs=output)
    model = Model(inputs=[inputs], outputs=output)
    return model

In [204]:
att_model3 = get_attention_model2()

In [205]:
att_model3.compile(optimizer='adam', loss='mae', metrics=['mse'])

In [ ]:
att_model_history3 = att_model3.fit(train_data_single, epochs=500,
#                                             steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            )#validation_steps=50

In [ ]:
# 预测
att_lstm_pre3=att_model3.predict(testx,verbose=1)

In [ ]:
attlstmy_modify3 = att_lstm_pre3*(att_lstm_pre3>=0)
attlstmr3=r2_score(testy,attlstmy_modify3)
attlstmr3

In [ ]:
# 保存注意力机制模型
att_model3.save('attlstm_model')

In [ ]:
# # 绘制训练集与验证集损失曲线
plt.figure(figsize=(16,8))
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.plot(att_model_history3.history['val_loss'],label=['attention_lstm_loss'])
plt.legend(loc='best')
plt.savefig("attention-LSTM损失曲线.jpg")
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))
ax = plt.subplot(111)
ax.set_title('attention-lstm-pm2.5')
ax.plot(testy[0:300, 0], "b", label="real", alpha=0.6)
ax.plot(attlstmy_modify3[0:300, 0], "r", label="pred", alpha=0.6)
plt.xlabel('time')
plt.savefig("attention-LSTM拟合图.jpg")
plt.legend()

In [239]:
label1=scaler.inverse_transform(testy)

In [ ]:
label1.max()

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)
preds = np.array(attlstmy_modify3)  # 转换为numpy形式
labels = np.array(testy)
# print(preds)
# print(labels.shape)
# print(preds.shape)
preds = preds.reshape((-1,1))
future_len = preds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(future_len):
    labels[:, k] = labels[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(future_len):
    preds[:, k] = preds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
labels.max()

In [ ]:
att_lstm_mae=mean_absolute_error(labels,preds)
att_lstm_mae

In [ ]:
att_lstm_mse=mean_squared_error(labels,preds)
att_lstm_mse

In [ ]:
att_mse_test=np.sum((preds-labels)**2)/len(labels) #跟数学公式一样的
att_mse_test

In [ ]:
att_rmse_test=att_mse_test ** 0.5
att_rmse_test

In [ ]:
att_mae_test=np.sum(np.absolute(preds-labels))/len(labels)
att_mae_test

In [233]:
att_model4 = get_attention_model2()

In [234]:
att_model4.compile(optimizer='adam', loss='mae')

In [ ]:
att_model_history4 = att_model4.fit(train_data_single, epochs=1500,
#                                             steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            )#validation_steps=50

In [ ]:
# 预测
att_lstm_pre4=att_model4.predict(testx,verbose=1)

In [ ]:
attlstmy_modify4 = att_lstm_pre4*(att_lstm_pre4>=0)
attlstmr4=r2_score(testy,attlstmy_modify4)
attlstmr4

双向lstm与注意力机制结合

In [253]:
def get_attention_bilstm():
    inputs=Input(shape=(120,7))
    lstm_units = 64
    # (batch_size, time_steps, INPUT_DIM) -> (batch_size, input_dim, lstm_units)
    lstm_out =Bidirectional(LSTM(lstm_units, return_sequences=True))(inputs)
#     model.add(Bidirectional(LSTM(256, activation='relu'), input_shape=(x_train_single.shape[-2],x_train_single.shape[-1]))
    attention_mul = attention_3d_block(lstm_out)
    # (batch_size, input_dim, lstm_units) -> (batch_size, input_dim*lstm_units)
    attention_mul = Flatten()(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
#     model = Model(inputs=inputs1, outputs=output)
    model = Model(inputs=[inputs], outputs=output)
    return model

In [254]:
att_bilstm_model=get_attention_bilstm()

In [255]:
att_bilstm_model.compile(optimizer='adam', loss='mae')

In [ ]:
att_bilstm_history = att_bilstm_model.fit(train_data_single, epochs=500,
#                                             steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            )#validation_steps=50

In [ ]:
# 预测
att_lstm_pre5=att_bilstm_model.predict(testx,verbose=1)

In [ ]:
attlstmy_modify5 = att_lstm_pre5*(att_lstm_pre5>=0)
attlstmr5=r2_score(testy,attlstmy_modify5)
attlstmr5

In [ ]:
att_bilstm_model.save('att_bilstm_model')

In [ ]:
model_in=Input(shape=(120,7))
lstm_out =Bidirectional(LSTM(32, return_sequences=True))(model_in)
hidden_states = lstm_out
hidden_size = int(hidden_states.shape[2])
h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
h_t

In [ ]:
hidden_states[:,-1,:]